In [91]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import gc
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error
import holidays

import warnings
warnings.filterwarnings('ignore')

In [92]:
all_data_df = pd.read_csv("Data.csv")
ss = pd.read_csv("SampleSubmission.csv")

In [93]:
weather_data = pd.read_excel("KalamClimateData.xlsx")

In [94]:
print(len(ss))
print(len(all_data_df))
print(len(weather_data))

6014
17140913
12228


In [95]:
weather_data.head()

,Date Time,Temperature (°C),Dewpoint Temperature (°C),U Wind Component (m/s),V Wind Component (m/s),Total Precipitation (mm),Snowfall (mm),Snow Cover (%)
0,2023-06-03 13:00:00,7.199731,-2.448126,-0.034195,0.060989,0.000377,0.0,99.972656
1,2023-06-03 14:00:00,6.000620,-2.597540,-0.080688,0.018585,0.000398,0.0,99.972656
2,2023-06-03 15:00:00,4.789972,-2.696951,0.057449,-0.352020,0.000418,0.0,99.972656
3,2023-06-03 16:00:00,3.158746,-2.879324,0.051529,-0.608948,0.000420,0.0,99.972656
4,2023-06-03 17:00:00,1.671426,-3.255606,0.056137,-0.739838,0.000420,0.0,99.972656


In [96]:
# Split 'Source' into 'consumer_device_X' and 'data_user_Y'
all_data_df[['consumer_device', 'data_user']] = all_data_df['Source'].str.extract(r'(consumer_device_\d+)_data_user_(\d+)')

# Display the updated DataFrame (optional)
all_data_df.head()

,date_time,v_red,current,power_factor,kwh,Source,v_blue,v_yellow,consumer_device,data_user
0,2024-07-22 18:20:00,137.65,0.08,0.72,0.000661,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1
1,2024-07-22 18:25:00,122.82,0.08,0.73,0.000598,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1
2,2024-07-22 18:30:00,119.70,0.08,0.74,0.000591,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1
3,2024-07-22 18:35:00,124.53,0.08,0.75,0.000623,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1
4,2024-07-22 18:40:00,134.84,0.08,0.74,0.000665,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1


In [97]:
# These are the devices that are not in the test SampleSubmission

devices_to_drop = ["consumer_device_3","consumer_device_5","consumer_device_11", "consumer_device_14",
                   "consumer_device_15", "consumer_device_17", "consumer_device_24",
                   "consumer_device_25","consumer_device_27","consumer_device_33","consumer_device_4","consumer_device_9"]

In [98]:
# Filter the DataFrame to include only rows where 'consumer_device' is in the 'devices_to_drop' list.
filtered_df = all_data_df[all_data_df['consumer_device'].isin(devices_to_drop)]

# Now 'filtered_df' contains only the rows you specified.  You can further process or save this DataFrame.
filtered_df.head()

,date_time,v_red,current,power_factor,kwh,Source,v_blue,v_yellow,consumer_device,data_user
327816,2023-10-01 14:35:00,169.26,3.50,0.73,0.036038,consumer_device_11_data_user_1,NaN,NaN,consumer_device_11,1
327817,2023-10-01 14:40:00,169.20,3.15,0.76,0.033755,consumer_device_11_data_user_1,NaN,NaN,consumer_device_11,1
327818,2023-10-01 14:45:00,168.38,2.58,0.73,0.026427,consumer_device_11_data_user_1,NaN,NaN,consumer_device_11,1
327819,2023-10-01 14:50:00,168.87,2.52,0.76,0.026952,consumer_device_11_data_user_1,NaN,NaN,consumer_device_11,1
327820,2023-10-01 14:55:00,168.30,2.47,0.75,0.025981,consumer_device_11_data_user_1,NaN,NaN,consumer_device_11,1


In [99]:
import pandas as pd

# Assuming 'all_data_df' is already defined as in your previous code.
# Convert 'Datetime' column to datetime objects if it's not already
all_data_df['date_time'] = pd.to_datetime(all_data_df['date_time'])

# Extract the date part
all_data_df['Date'] = all_data_df['date_time'].dt.date

# Group by 'Source' and 'Date', then sum the specified columns for each group
aggregated_data = all_data_df.groupby(['Source', 'Date']).agg({
    'kwh': 'sum',
    'v_red': 'sum',
    'current': 'sum',
    'power_factor': 'sum',
    'v_blue': 'sum',
    'v_yellow': 'sum'
}).reset_index()

# Display the aggregated data
print(aggregated_data.head())

                           Source        Date       kwh     v_red  current  \
0  consumer_device_10_data_user_1  2024-07-22  0.024330   6743.08     5.43   
1  consumer_device_10_data_user_1  2024-07-23  0.103560  29279.68    17.42   
2  consumer_device_10_data_user_1  2024-07-24  0.137543  28823.31    21.33   
3  consumer_device_10_data_user_1  2024-07-25  0.121011  29123.31    23.83   
4  consumer_device_10_data_user_1  2024-07-26  0.000000   9264.52     0.00   

   power_factor  v_blue  v_yellow  
0         26.12     0.0       0.0  
1         98.78     0.0       0.0  
2        115.17     0.0       0.0  
3        102.60     0.0       0.0  
4          0.00     0.0       0.0  


In [100]:
aggregated_data.head()

,Source,Date,kwh,v_red,current,power_factor,v_blue,v_yellow
0,consumer_device_10_data_user_1,2024-07-22,0.024330,6743.08,5.43,26.12,0.0,0.0
1,consumer_device_10_data_user_1,2024-07-23,0.103560,29279.68,17.42,98.78,0.0,0.0
2,consumer_device_10_data_user_1,2024-07-24,0.137543,28823.31,21.33,115.17,0.0,0.0
3,consumer_device_10_data_user_1,2024-07-25,0.121011,29123.31,23.83,102.60,0.0,0.0
4,consumer_device_10_data_user_1,2024-07-26,0.000000,9264.52,0.00,0.00,0.0,0.0


In [101]:
# Find the minimum and maximum date_time values
min_date = aggregated_data['Date'].min()
max_date = aggregated_data['Date'].max()

print(f"Minimum date_time: {min_date}")
print(f"Maximum date_time: {max_date}")

Minimum date_time: 2023-06-24
Maximum date_time: 2024-09-23


In [102]:
aggregated_data.shape

(59728, 8)

In [103]:
aggregated_data.isnull().sum()

Source          0
Date            0
kwh             0
v_red           0
current         0
power_factor    0
v_blue          0
v_yellow        0
dtype: int64

In [104]:
# # Fill missing date values with 0 kwh

# # Create a date range
# date_rng = pd.date_range(start=min_date, end=max_date, freq='D')

# # Create an empty DataFrame to store the complete data
# complete_data = pd.DataFrame()

# # Iterate through each unique 'Source'
# for source in aggregated_data['Source'].unique():
#     # Extract data for the current 'Source'
#     source_data = aggregated_data[aggregated_data['Source'] == source].copy()

#     # Convert the source data Date to match the type of date_rng
#     source_data['Date'] = pd.to_datetime(source_data['Date'])

#     # Create a complete date range for the current 'Source'
#     source_date_rng = pd.DataFrame({'Date': date_rng})
#     source_date_rng['Source'] = source

#     # Merge with the existing data, filling missing 'kwh' values with 0
#     source_data = pd.merge(source_date_rng, source_data, on=['Date', 'Source'], how='left')
#     source_data['kwh'] = source_data['kwh'].fillna(0)

#     # Append to the complete data
#     complete_data = pd.concat([complete_data, source_data], ignore_index=True)

In [105]:
all_data_df.head()

,date_time,v_red,current,power_factor,kwh,Source,v_blue,v_yellow,consumer_device,data_user,Date
0,2024-07-22 18:20:00,137.65,0.08,0.72,0.000661,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1,2024-07-22
1,2024-07-22 18:25:00,122.82,0.08,0.73,0.000598,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1,2024-07-22
2,2024-07-22 18:30:00,119.70,0.08,0.74,0.000591,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1,2024-07-22
3,2024-07-22 18:35:00,124.53,0.08,0.75,0.000623,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1,2024-07-22
4,2024-07-22 18:40:00,134.84,0.08,0.74,0.000665,consumer_device_10_data_user_1,NaN,NaN,consumer_device_10,1,2024-07-22


In [106]:
def add_time_features(df):
    """
    Adds time-based features (month and weekday) to the DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame with a 'Date' column.

    Returns:
        pd.DataFrame: The DataFrame with added 'month' and 'weekday' columns.
    """
    # Ensure 'Date' is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df['Date']):
        df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract month and weekday as integers
    df["month"] = df['Date'].dt.month
    df["weekday"] = df['Date'].dt.weekday  # Monday=0, Sunday=6
    
    return df

In [107]:
aggregated_data = add_time_features(aggregated_data)
aggregated_data.head()

,Source,Date,kwh,v_red,current,power_factor,v_blue,v_yellow,month,weekday
0,consumer_device_10_data_user_1,2024-07-22,0.024330,6743.08,5.43,26.12,0.0,0.0,7,0
1,consumer_device_10_data_user_1,2024-07-23,0.103560,29279.68,17.42,98.78,0.0,0.0,7,1
2,consumer_device_10_data_user_1,2024-07-24,0.137543,28823.31,21.33,115.17,0.0,0.0,7,2
3,consumer_device_10_data_user_1,2024-07-25,0.121011,29123.31,23.83,102.60,0.0,0.0,7,3
4,consumer_device_10_data_user_1,2024-07-26,0.000000,9264.52,0.00,0.00,0.0,0.0,7,4


In [108]:
def add_holiday_indicator(df, country='Pakistan'):
    """
    Adds a holiday indicator column to the DataFrame based on the country's holidays.

    Parameters:
        df (pd.DataFrame): The input DataFrame with a 'Date' column.
        country (str): The country for which holidays are to be checked (default: 'Pakistan').

    Returns:
        pd.DataFrame: The DataFrame with an added 'is_holiday' column.
    """
    # Ensure 'Date' is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df['Date']):
        df['Date'] = pd.to_datetime(df['Date'])
    
    # Load holidays for the specified country
    country_holidays = holidays.CountryHoliday(country)
    
    # Add holiday indicator
    df['is_holiday'] = df['Date'].apply(lambda x: x in country_holidays)
    
    return df

In [109]:
aggregated_data = add_holiday_indicator(aggregated_data)
aggregated_data.head()

,Source,Date,kwh,v_red,current,power_factor,v_blue,v_yellow,month,weekday,is_holiday
0,consumer_device_10_data_user_1,2024-07-22,0.024330,6743.08,5.43,26.12,0.0,0.0,7,0,False
1,consumer_device_10_data_user_1,2024-07-23,0.103560,29279.68,17.42,98.78,0.0,0.0,7,1,False
2,consumer_device_10_data_user_1,2024-07-24,0.137543,28823.31,21.33,115.17,0.0,0.0,7,2,False
3,consumer_device_10_data_user_1,2024-07-25,0.121011,29123.31,23.83,102.60,0.0,0.0,7,3,False
4,consumer_device_10_data_user_1,2024-07-26,0.000000,9264.52,0.00,0.00,0.0,0.0,7,4,False


In [110]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Function to process and forecast per unique consumer_device_x and data_user_y
def forecast_arima(all_data, forecast_horizon=30, output_template=None):
    # Convert Date column to datetime format
    all_data['Date'] = pd.to_datetime(all_data['Date'])

    # Extract consumer_device_x and data_user_y
    all_data[['consumer_device', 'data_user']] = all_data['Source'].str.extract(r'consumer_device_(\d+)_data_user_(\d+)')

    # Ensure data is sorted by consumer_device, data_user, and Date
    all_data = all_data.sort_values(by=['consumer_device', 'data_user', 'Date'])

    # Store forecasts
    forecast_results = []

    # Process each unique consumer_device_x and data_user_y combination
    for (consumer_device, data_user), group in all_data.groupby(["consumer_device", "data_user"]):
        # Set Date as index
        group = group.set_index("Date")

        # Ensure data is in the correct format
        group = group.asfreq('D').fillna(method='ffill')  # Fill missing dates with last known value

        # Fit ARIMA model
        try:
            model = ARIMA(group["kwh"], order=(5, 1, 0))  # ARIMA(5,1,0) as a baseline
            fitted_model = model.fit()

            # Forecast for the next forecast_horizon days
            forecast_dates = pd.date_range(start=group.index[-1] + pd.Timedelta(days=1),
                                           periods=forecast_horizon, freq='D')
            forecast_values = fitted_model.forecast(steps=forecast_horizon)

            # Store results in required format
            forecast_df = pd.DataFrame({
                "ID": [f"{date.strftime('%Y-%m-%d')}_consumer_device_{consumer_device}_data_user_{data_user}"
                        for date in forecast_dates],
                "kwh": forecast_values
            })

            forecast_results.append(forecast_df)

        except Exception as e:
            print(f"Error processing {consumer_device}_{data_user}: {e}")

    # Combine all forecasts into a single DataFrame
    forecast_df = pd.concat(forecast_results, ignore_index=True)

    # If an output template is provided, align the output format
    if output_template is not None:
        output_template = output_template.drop(columns=['kwh'], errors='ignore')
        final_output = output_template.merge(forecast_df, on='ID', how='left').fillna(0)
    else:
        final_output = forecast_df

    return final_output


In [111]:
forecast = forecast_arima(all_data=aggregated_data, forecast_horizon=30, output_template=ss)

In [112]:
forecast.head()

,ID,kwh
0,2024-09-24_consumer_device_12_data_user_1,0.156996
1,2024-09-25_consumer_device_12_data_user_1,0.157035
2,2024-09-26_consumer_device_12_data_user_1,0.206451
3,2024-09-27_consumer_device_12_data_user_1,0.188690
4,2024-09-28_consumer_device_12_data_user_1,0.166484


In [113]:
# prompt: does forecast["kwh"] contain nans if so replace with 0

# Check for NaN values in the 'kwh' column and replace them with 0
forecast["kwh"] = forecast["kwh"].fillna(0)


In [114]:
len(aggregated_data), len(forecast), len(ss)

(59728, 6014, 6014)

In [115]:
forecast.to_csv("forecast_with_all_v_current_powerfactor_weekday_month_holiday.csv", index = False)